# Asset Hierarchy Migration

**Prerequisite**:

- Access to a CDF Project.
- Know how to install and setup Python.
- Launch a Python notebook.



In this tutorial, we will show you how to migrate an asset hierarchy to a data model representing the same hierarchy in CDF.

This tutorial is also a good demonstration of the width of capabilities for neat, going from extracting data,
creating a data model, export the data model, and load the data.

## Extract Data from Asset Hierarchy

We will start by extracting the data from an existing asset hierarchy. 

The example we will use in this tutorial is an asset hierarchy for pumps, shown below in CDF's classic Data Exploration

<img src="../artifacts/figs/asset_hierarchy_lift_pump_stations.png" width="400">

In [1]:
from cognite.neat import get_cognite_client
from cognite.neat.graph import extractors, NeatGraphStore

We start by importing the neat extractors, NeatGraphStore, and a utility method for getting a neat client.

In [2]:
store = NeatGraphStore.from_memory_store()

The NeatGraphStore is a triple store that can store both data as well as schemas/data-models.

This store we can populate by extracting from one of the supported sources for neat. 

To see which extractors are available we print the `extractors` module.

In [3]:
extractors

<module 'cognite.neat.graph.extractors' from 'C:\\Users\\AndersAlbert\\Projects\\internal\\neat\\cognite\\neat\\graph\\extractors\\__init__.py'>

At the top of the list we see the AssetsExtractor which is what we are looking for

In [4]:
extractors.AssetsExtractor

cognite.neat.graph.extractors._classic_cdf._assets.AssetsExtractor

We see that there is a `.from_hierarchy` fatory method that fits well with what we want to do

In [5]:
client = get_cognite_client()

Found .env file in repository root. Loaded variables from .env file.


Note that the utility function `get_cognite_client` will prompt us for credentials if it doesn't find a `.env` file in the repo root.

In [6]:
asset_extractor = extractors.AssetsExtractor.from_hierarchy(client, root_asset_external_id="lift_pump_stations:root")

The asset extractor is now ready and can be written to the store.

In [7]:
store.write(asset_extractor)

We can inspect the store to see what changes have been applied to it:

In [8]:
store

## Infering Data Model from Data

To infer a data model from the data we will use an importer

In [9]:
from cognite.neat.rules import importers

In [10]:
importers

<module 'cognite.neat.rules.importers' from 'C:\\Users\\AndersAlbert\\Projects\\internal\\neat\\cognite\\neat\\rules\\importers\\__init__.py'>

We see that there are multiple importers availabe, but we will use the `InferenceImporter`

In [11]:
importers.InferenceImporter

cognite.neat.rules.importers._inference2rules.InferenceImporter

In [12]:
importer = importers.InferenceImporter.from_graph_store(store)

Then we use the `.to_rules` method to convert the data into `Rules` which is `Neat`'s format for data models.

In [13]:
rules, issues = importer.to_rules()

First we check if there were any issues with creating the rules, and we find one warning. This warning 
is that there is a property, `Shape__Length`, with a double underscore which is not recommended. However,
wecan continue.

In [14]:
issues

IssueList([MoreThanOneNonAlphanumericCharacterWarning(field_name='property', value='Shape__Length')])

Then, we inspect the classes found along with the properties

In [15]:
rules.classes

SheetList[InformationClass](data=[InformationClass(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), name=None, description=None, parent=None, reference=Url('http://purl.org/cognite/neat#Asset'), match_type='exact', comment='Inferred from knowledge graph, where this class has <245> instances')])

In [16]:
rules.properties

SheetList[InformationProperty](data=[InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='name', name=None, description=None, value_type=String(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#name'), match_type=None, transformation=inferred:Asset(inferred:name), comment='Class <Asset> has property <name> with value type <string> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='external_id', name=None, description=None, value_type=String(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#external_id'), match_type=None, transformation=inferred:Asset(inferred:external_id), comment='Class <Asset> has property <external_id> with value type <string> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='created_time', name=None, description=None, value_type=DateTime(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#created_time'), match_type=None, transformation=inferred:Asset(inferred:created_time), comment='Class <Asset> has property <created_time> with value type <dateTime> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='last_updated_time', name=None, description=None, value_type=DateTime(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#last_updated_time'), match_type=None, transformation=inferred:Asset(inferred:last_updated_time), comment='Class <Asset> has property <last_updated_time> with value type <dateTime> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='DesignPointFlowGPM', name=None, description=None, value_type=Double(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#DesignPointFlowGPM'), match_type=None, transformation=inferred:Asset(inferred:DesignPointFlowGPM), comment='Class <Asset> has property <DesignPointFlowGPM> with value type <double> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='DesignPointHeadFT', name=None, description=None, value_type=Double(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#DesignPointHeadFT'), match_type=None, transformation=inferred:Asset(inferred:DesignPointHeadFT), comment='Class <Asset> has property <DesignPointHeadFT> with value type <double> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='Enabled', name=None, description=None, value_type=Double(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#Enabled'), match_type=None, transformation=inferred:Asset(inferred:Enabled), comment='Class <Asset> has property <Enabled> with value type <double> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='HighHeadShutOff', name=None, description=None, value_type=Double(), min_count=None, max_count=1, default=None, reference=Url('http://purl.org/cognite/neat#HighHeadShutOff'), match_type=None, transformation=inferred:Asset(inferred:HighHeadShutOff), comment='Class <Asset> has property <HighHeadShutOff> with value type <double> which occurs <245> times in the graph', inherited=False), InformationProperty(validators_to_skip=set(), class_=class(prefix=inferred,suffix=Asset), property_='LowHeadFT', name=None, description=None, v

We notice that for example the `PumpModel` is both an integer and a string, as the `Inference` found data of both types.

We can inspect the comment from the `Inference` type:

In [17]:
rules.properties.data[24].comment

'Class <Asset> has property <PumpModel> with value type <integer> which occurs <6> times in the graph, with value type <string> which occurs <140> times in the graph'

And we see that this is most likely a string as that occured much more for this field in the graph than the integer.

## Exporting Data Model

Lets export our newly created data model to CDF

In [18]:
from cognite.neat.rules import exporters

In [19]:
exporters

<module 'cognite.neat.rules.exporters' from 'C:\\Users\\AndersAlbert\\Projects\\internal\\neat\\cognite\\neat\\rules\\exporters\\__init__.py'>

To export the data model we use the `DMSExporter`.

In [20]:
exporters.DMSExporter

cognite.neat.rules.exporters._rules2dms.DMSExporter

To export the rules, we need them in the DMS format, however, the `rules` we have are in Information format.

In [21]:
type(rules)

cognite.neat.rules.models.information._rules.InformationRules

The information rules is used to model the information, while the DMS format is one of the implementation formats that Neat supports.

Neat has an out-of-the box conversion from Information to DMS formats, however, it does not, for example, set indexes.

In [22]:
dms_rules = rules.as_dms_architect_rules()

C:\Users\AndersAlbert\AppData\Local\pypoetry\Cache\virtualenvs\cognite-neat-WszCo0Uu-py3.12\Lib\site-packages\pydantic\main.py:176: MoreThanOneNonAlphanumericCharacterWarning: ('property', 'Shape__Length')
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [23]:
dms_rules.metadata

DMSMetadata(validators_to_skip=set(), data_model_type=<DataModelType.enterprise: 'enterprise'>, schema_=<SchemaCompleteness.partial: 'partial'>, extension=<ExtensionCategory.addition: 'addition'>, space='inferred', name='Inferred Model', description=None, external_id='inferred_model', version='inferred', creator=['NEAT'], created=datetime.datetime(2024, 7, 9, 7, 30, 24, 429571), updated=datetime.datetime(2024, 7, 9, 7, 30, 24, 429571))

In [24]:
dms_rules.views

SheetList[DMSView](data=[DMSView(validators_to_skip=set(), view=view(prefix=inferred,suffix=Asset,version=inferred), name=None, description=None, implements=[], reference=Url('http://purl.org/cognite/neat#Asset'), filter_=None, in_model=True, class_=class(prefix=inferred,suffix=Asset))])

In [25]:
dms_rules.containers

SheetList[DMSContainer](data=[DMSContainer(validators_to_skip=set(), container=container(prefix=inferred,suffix=Asset), name=None, description=None, reference=None, constraint=None, class_=class(prefix=inferred,suffix=Asset))])

If you want to modify the DMS rules it is recommented that you export them using the `ExcelExporter`, modify the resulting spreadsheet
and import it using the `ExcelImporter`. For this tutorial, we are happy with the out-of-the-box DMS rules, so we just pass
the `InformationRules` into the DMS exporter which will automatically do the conversion.

In [26]:
exporter = exporters.DMSExporter()

In [27]:
result = exporter.export_to_cdf(rules, client)

C:\Users\AndersAlbert\AppData\Local\pypoetry\Cache\virtualenvs\cognite-neat-WszCo0Uu-py3.12\Lib\site-packages\pydantic\main.py:176: MoreThanOneNonAlphanumericCharacterWarning: ('property', 'Shape__Length')
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [28]:
result

[UploadResult(name='spaces', error_messages=[], issues=[], created={'inferred'}, deleted=set(), changed=set(), unchanged=set(), skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='containers', error_messages=[], issues=[], created={ContainerId(space='inferred', external_id='Asset')}, deleted=set(), changed=set(), unchanged=set(), skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='views', error_messages=[], issues=[], created={ViewId(space='inferred', external_id='Asset', version='inferred')}, deleted=set(), changed=set(), unchanged=set(), skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set()),
 UploadResult(name='data_models', error_messages=[], issues=[], created={DataModelId(space='inferred', external_id='inferred_model', version='inferred')}, deleted=set(), changed=set(), unchanged=set(), skipped=set(), failed_created=set(), failed_upserted=set(), failed_changed=set(), failed_deleted=set())]

We see the data model was succesfully created.

## Populating Data Model

To populate the data model in CDF, we use a loader.

In [29]:
from cognite.neat.graph import loaders

In [30]:
loaders

<module 'cognite.neat.graph.loaders' from 'C:\\Users\\AndersAlbert\\Projects\\internal\\neat\\cognite\\neat\\graph\\loaders\\__init__.py'>

In [31]:
loaders.DMSLoader

cognite.neat.graph.loaders._rdf2dms.DMSLoader

To load the data from the graph store, we add the rules to the store

In [32]:
store.add_rules(rules)

In [33]:
store

This is necessary for the store to be ready to load data to an extrenal system. Note that by adding the `rules` to the store
the prefixes has been updated to match the rules object. This is how the loader knows which triples to fetch from the store.

First, we ensure the instance space exists.

In [34]:
from cognite.client import data_modeling as dm

In [35]:
created = client.data_modeling.spaces.apply(dm.SpaceApply("sp_pump_station"))
created

<Space(space='sp_pump_station') at 0x246db9a5c10>

We can now use the loader to populate the data model in CDF.

Note that the `DMSLoader` requires that we have the `DMSRules` format, so we pass in the DMS rules we created above.

Alternatively, we can create the loader by passing in a data model ID.

In [36]:
loader = loaders.DMSLoader.from_rules(dms_rules, store, instance_space="sp_pump_station")

In [37]:
result = loader.load_into_cdf(client)

In [38]:
result

[UploadResult(name='Nodes', error_messages=[], issues=[], created={NodeId(space='sp_pump_station', external_id='Asset_1599542126336595'), NodeId(space='sp_pump_station', external_id='Asset_1328367249298685'), NodeId(space='sp_pump_station', external_id='Asset_8536226378041620'), NodeId(space='sp_pump_station', external_id='Asset_5687470951999200'), NodeId(space='sp_pump_station', external_id='Asset_2738034707487198'), NodeId(space='sp_pump_station', external_id='Asset_2011390888860506'), NodeId(space='sp_pump_station', external_id='Asset_1717136324291166'), NodeId(space='sp_pump_station', external_id='Asset_673071391885663'), NodeId(space='sp_pump_station', external_id='Asset_3009457875737229'), NodeId(space='sp_pump_station', external_id='Asset_3253702221176761'), NodeId(space='sp_pump_station', external_id='Asset_2041897821048926'), NodeId(space='sp_pump_station', external_id='Asset_5773835011768211'), NodeId(space='sp_pump_station', external_id='Asset_791284880355712'), NodeId(space='sp_pump_station', external_id='Asset_720243198469901'), NodeId(space='sp_pump_station', external_id='Asset_6276047304545772'), NodeId(space='sp_pump_station', external_id='Asset_779363703909879'), NodeId(space='sp_pump_station', external_id='Asset_765927258989033'), NodeId(space='sp_pump_station', external_id='Asset_8149656736332002'), NodeId(space='sp_pump_station', external_id='Asset_3912512319616642'), NodeId(space='sp_pump_station', external_id='Asset_2614391887415841'), NodeId(space='sp_pump_station', external_id='Asset_2622548696369535'), NodeId(space='sp_pump_station', external_id='Asset_4030282795820257'), NodeId(space='sp_pump_station', external_id='Asset_597713956124653'), NodeId(space='sp_pump_station', external_id='Asset_1036635642295968'), NodeId(space='sp_pump_station', external_id='Asset_6264128498357271'), NodeId(space='sp_pump_station', external_id='Asset_4487513710880340'), NodeId(space='sp_pump_station', external_id='Asset_690992856868092'), NodeId(space='sp_pump_station', external_id='Asset_4356678455013717'), NodeId(space='sp_pump_station', external_id='Asset_4139420867004251'), NodeId(space='sp_pump_station', external_id='Asset_3718586005996830'), NodeId(space='sp_pump_station', external_id='Asset_7412273070008116'), NodeId(space='sp_pump_station', external_id='Asset_2534347363555646'), NodeId(space='sp_pump_station', external_id='Asset_8906381697072008'), NodeId(space='sp_pump_station', external_id='Asset_2648639705661733'), NodeId(space='sp_pump_station', external_id='Asset_8786769086021865'), NodeId(space='sp_pump_station', external_id='Asset_8855980446983123'), NodeId(space='sp_pump_station', external_id='Asset_6996457070033380'), NodeId(space='sp_pump_station', external_id='Asset_5991677408633367'), NodeId(space='sp_pump_station', external_id='Asset_1424210982323954'), NodeId(space='sp_pump_station', external_id='Asset_1304177008138755'), NodeId(space='sp_pump_station', external_id='Asset_413299123404753'), NodeId(space='sp_pump_station', external_id='Asset_8724043116543799'), NodeId(space='sp_pump_station', external_id='Asset_3543366413287754'), NodeId(space='sp_pump_station', external_id='Asset_2383890056251078'), NodeId(space='sp_pump_station', external_id='Asset_3322086940842206'), NodeId(space='sp_pump_station', external_id='Asset_862846106956486'), NodeId(space='sp_pump_station', external_id='Asset_3282403756294926'), NodeId(space='sp_pump_station', external_id='Asset_1602743241841442'), NodeId(space='sp_pump_station', external_id='Asset_6499093917037259'), NodeId(space='sp_pump_station', external_id='Asset_7062323820650437'), NodeId(space='sp_pump_station', external_id='Asset_2109398666822135'), NodeId(space='sp_pump_station', external_id='Asset_8808794232286601'), NodeId(space='sp_pump_station', external_id='Asset_8628246215971298'), NodeId(space='sp_pump_station', external_id='Asset_6781772664404616'), NodeId(space='sp_pump_station', external_id='Asset_4161802645953289'), NodeId(space='sp_pump_station', exter

As we see from the result above, Neat has created 245 Nodes in the new data model.

## Results

We can now go into CDF and inspect the results. Looking at the data model we created, we can see the schema for
the inferred Asset

<img src="../artifacts/figs/asset_hierarchy_lift_pump_stations_dms.png" width="400">

Furthermore, we can inspect the populated nodes in this Asset schema

<img src="../artifacts/figs/asset_hierarchy_lift_pump_stations_populated.png" width="1000">

## Final Remarks

* In this tutorial, we used the in-memory version of the Neat store. This works well for small examples, like the toy example here, but for larger asset hierarchies we need to likely use a faster triple store such as `GraphDB` or `Oxigraph`. These are also available in Neat, but might require extra dependencies.
* This can be considered the first step of a full migration. At least two related problems may remain
    1. First, we might want to infer a more specific type than `Asset`, for example, `Pump` and `LiftStation`. This means adding information that is not explicitly set in the existing Asset Hierarchy. The type might be implicitly defined from the level in the hierarchy, or for example, the external ID of the asset.
    2. We might want to map the inferred model onto an existing data model. It this case the existing model would be an `EnterpriseModel` and the inferred model we obtained here would be a `Source` model. 
